In [ ]:
!pip install wandb -qqq
!apt install tree

     |████████████████████████████████| 1.9 MB 5.0 MB/s 
     |████████████████████████████████| 182 kB 85.8 MB/s 
     |████████████████████████████████| 173 kB 73.3 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 168 kB 84.8 MB/s 
     |████████████████████████████████| 168 kB 82.1 MB/s 
     |████████████████████████████████| 166 kB 115.3 MB/s 
     |████████████████████████████████| 166 kB 89.0 MB/s 
     |████████████████████████████████| 162 kB 102.0 MB/s 
     |████████████████████████████████| 162 kB 97.4 MB/s 
     |████████████████████████████████| 158 kB 104.9 MB/s 
     |████████████████████████████████| 157 kB 101.7 MB/s 
     |████████████████████████████████| 157 kB 70.3 MB/s 
     |████████████████████████████████| 157 kB 91.2 MB/s 
     |████████████████████████████████| 157 kB 91.9 MB/s 
     |████████████████████████████████| 157 kB 91.4 MB/s 
     |████████████████████████████████| 157 kB 114.2 MB/s 
     |██████

In [ ]:
# !wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import random
import torch
import torchvision
from torch.utils.data import TensorDataset
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from math import sqrt
import math, time
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error
from torch.utils.data import DataLoader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## RAW DATA VERSIONING

The raw dataset of Apple stock prices was uploaded to the W&B platform to keep and versioning.

In [ ]:
# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def load():
    """
    # Load the data
    """

    path="/content/drive/My Drive/Colab Notebooks/DataCraft_notebook/"
    data=pd.read_csv(path+"Data_Apple.csv")
    data=data.copy()
    return data
dataset=load()

with wandb.init(project="DSS",job_type="load-data", name="rawdata-load_APPLE"):
  artifact = wandb.Artifact( 'Apple','Data')
  table = wandb.Table(columns=list(dataset.columns), data=dataset.to_numpy())
  artifact.add(table, "rawdata")
  wandb.log_artifact(artifact)

## MISSING DATA HANDLING

In this section, missing values were filled by using 10 days moving average values. 

In [ ]:
def preprocess(data):
    """
    ## Prepare the data
    """
    data["Change %"]=data["Change"].str.replace("%","") #edit change column to make it numeric
    data["Change %"]=pd.to_numeric(data["Change %"]) 
    data['Date'] = pd.to_datetime(data.Date)
    data['Close']=0
    for i in range(1, len(data)):
      data.loc[i, 'Close'] = data.loc[i-1, 'Open'] #create close column
    data['Volume']=data['Vol'].apply(lambda x: float(x[:-1])).
    return data

In [ ]:
#create an empty dataset which includes all days in the time period 
def create_date_table(start='2017-01-03', end='2022-10-31'):
   df = pd.DataFrame({"Date": pd.date_range(start, end)})
   return df

In [ ]:
#fill the empty data with 10 days moving average.
def fill_missing(data):
  data = data[["Date","Price","Open","High","Low","Change %","Volume"]]
  date_data= create_date_table()
  df = date_data.merge(data, left_on='Date', right_on='Date', how='left')
  df=df[["Date","Price","Open","High","Low","Change %","Volume"]]
  for column in df.columns:
    if df[column].dtypes!='<M8[ns]':
      df[column+'_MA']=df[column].rolling(10, min_periods=1).mean()
      df[column].fillna(df[column+'_MA'], inplace=True)
  df['Date']=df['Date'].astype(str)    
  filled_df = df[["Date","Price","Open","High","Low","Change %","Volume"]]
  return filled_df

In [ ]:
#log the new version of the dataset to W&B
#the new version the dataset includes the missing data.
def preprocess_and_log():

    with wandb.init(project="DSS") as r:
         
        # ✔️ declare which artifact we'll be using
        artifact = r.use_artifact('metu_datacraft/DSS/Apple:v0', type='Data')
        table = artifact.get('rawdata')
        dataset= {"Date": table.get_column("Date"),"Price":table.get_column("Price"),"Open":table.get_column("Open"),
                  "High":table.get_column("High"), "Low":table.get_column("Low") , "Vol": table.get_column("Vol."),
                  "Change":table.get_column("Change %")}

        data = pd.DataFrame(dataset)
        preprocessed_data=preprocess(data)
        filled_data=fill_missing(preprocessed_data)
        
        with wandb.init(project="DSS",job_type="preprocess-data",name="filleddata-load-APPLE"):
          artifact = wandb.Artifact( 'Apple','Data')
          table = wandb.Table(columns=list(filled_data.columns), data=filled_data.to_numpy())
          artifact.add(table, "filled_data")
          wandb.log_artifact(artifact)
preprocess_and_log()

wandb:   1 of 1 files downloaded.  


##DATA VERSION 3 (Removing weekends)


In this section, the filled days of weekends were removed from the dataset. And it was versioned to the W%B platform as a 3rd version of the dataset.

In [ ]:
with wandb.init(project="DSS") as r:
         
        # ✔️ declare which artifact we'll be using
        artifact = r.use_artifact('metu_datacraft/DSS/Apple:v1', type='Data')
        table = artifact.get('filled_data')
        dataset= {"Date": table.get_column("Date"),"Price":table.get_column("Price"),"Open":table.get_column("Open"),
                  "High":table.get_column("High"), "Low":table.get_column("Low") , "Volume": table.get_column("Volume"),
                  "Change":table.get_column("Change %")}
        data = pd.DataFrame(dataset)

        #remove weekends from dataset
        data['Date']=pd.to_datetime(data['Date'])
        data=data[(data['Date'].dt.day_name()!='Saturday') & (data['Date'].dt.day_name()!='Sunday')]
        data['Date']=data['Date'].astype(str)  

        #add to WB
        with wandb.init(project="DSS",job_type="preprocess-data",name="weekdata-load-APPLE"):
           artifact = wandb.Artifact( 'Apple','Data')
           table = wandb.Table(columns=list(data.columns), data=data.to_numpy())
           artifact.add(table, "weekday_data")
           wandb.log_artifact(artifact)


wandb:   1 of 1 files downloaded.  
